In [1]:
from bs4 import BeautifulSoup
import requests
import re
import numpy as np
from sklearn.base import TransformerMixin
from sklearn.pipeline import make_pipeline, make_union
from sklearn.tree import DecisionTreeClassifier
from sklearn.cross_validation import train_test_split

In [2]:
urls_page = BeautifulSoup(requests \
            .get('http://rosettacode.org/wiki/Category:Solutions_by_Programming_Task') \
            .content, 'html.parser').find_all('div', {'class' : 'mw-content-ltr'})[-1]

In [4]:
#urls_page.find('div', {'class' : 'mw-content-ltr'})
urls = re.findall(r'href="(.*)" ', str(urls_page))[:15]
pages = [BeautifulSoup(requests.get('http://rosettacode.org'+url) \
                       .content, 'html.parser') for url in urls]

In [ ]:
#doors = requests.get('http://rosettacode.org/wiki/100_doors')

# doors = BeautifulSoup(requests.get('http://rosettacode.org/wiki/100_doors') \
#                       .content, 'html.parser')



In [ ]:
# doors.prettify

In [34]:
langs = ['c', 'csharp', 'lisp', 'clojure', 'haskell', 
         'java', 'javascript', 'ocaml', 'perl', 'php', 
         'python', 'ruby', 'scala', 'scheme']

# lang_dict = {lang : '' for lang in langs}
lang_X = []
lang_y = []

for page in pages:
    for string in page.find_all('pre'):
        for lang in langs:
            if re.search(r'"'+ lang + ' hi', str(string)):
                # lang_dict[lang] = lang_dict[lang] + string.get_text()
                lang_X.append(string.get_text())
                lang_y.append(lang)
#print(lang_dict)

In [35]:
lang_X

['(defun rep (n x)   (if (zp n)       nil       (cons x             (rep (- n 1) x))))\xa0(defun toggle-every-r (n i bs)   (if (endp bs)       nil       (cons (if (zp i)                 (not (first bs))                 (first bs))             (toggle-every-r n (mod (1- i) n) (rest bs)))))\xa0(defun toggle-every (n bs)   (toggle-every-r n (1- n) bs))\xa0(defun 100-doors (i doors)   (if (zp i)       doors       (100-doors (1- i) (toggle-every i doors))))',
 '#include <stdio.h>\xa0int main(){  char is_open[100] = { 0 };  int pass, door;\xa0  /* do the 100 passes */  for (pass = 0; pass < 100; ++pass)    for (door = pass; door < 100; door += pass+1)      is_open[door] = !is_open[door];\xa0  /* output the result */  for (door = 0; door < 100; ++door)    printf("door #%d is\xa0%s.\\n", door+1, (is_open[door]? "open" : "closed"));\xa0  return 0;}',
 '#include <stdio.h>\xa0#define NUM_DOORS 100\xa0int main(int argc, char *argv[]){  int is_open[NUM_DOORS] = { 0 } ;  int * doorptr, * doorlimit =

In [36]:
class FunctionFeaturizer(TransformerMixin):
    def __init__(self, *featurizers):
        self.featurizers = featurizers
        
    def fit(self, X, y=None):
        '''Returns self after transformations'''
        return self
    
    def transform(self, X):
        '''Given a list of original data, return a list of feature vectors'''
        feature_vectors = []
        for x in X:
            feature_vector = [f(x) for f in self.featurizers]
            feature_vectors.append(feature_vector)
        return np.array(feature_vectors)

In [176]:
# chars = ['(', ')', '{', '}', '[', ']', '<', '>', '$', '/', ' ', '\t']
# frequencies = [lambda text: text.count(char)/len(text) for char in chars]
def paren_freq(text):
    '''checks the frequency of the ( character'''
    return text.count('(')/len(text)

def thesis_freq(text):
    '''checks the frequency of the ) character'''
    return text.count(')')/len(text)

def brac_freq(text):
    '''checks the frequency of the ) character'''
    return text.count('[')/len(text)

def ket_freq(text):
    '''checks the frequency of the ) character'''
    return text.count(']')/len(text)
# print(*frequencies)
def lesser_freq(text):
    '''checks the frequency of the ) character'''
    return text.count('<')/len(text)
def greater_freq(text):
    '''checks the frequency of the ) character'''
    return text.count('>')/len(text)
def slash_freq(text):
    '''checks the frequency of the ) character'''
    return text.count('/')/len(text)
def dollar_freq(text):
    '''checks the frequency of the ) character'''
    return text.count('$')/len(text)
def double_quote_freq(text):
    '''checks the frequency of the ) character'''
    return text.count('"')/len(text)
def space_freq(text):
    '''checks the frequency of the ) character'''
    return text.count(' ')/len(text)
def semi_freq(text):
    '''checks the frequency of the ) character'''
    return text.count(';')/len(text)
def colon_freq(text):
    '''checks the frequency of the ) character'''
    return text.count(':')/len(text)
def at_freq(text):
    '''checks the frequency of the ) character'''
    return text.count('@')/len(text)
def star_freq(text):
    '''checks the frequency of the ) character'''
    return text.count('*')/len(text)
def pipe_freq(text):
    return text.count('|')/len(text)

def if_defn(text):
    if re.search(r'defn', text):
        return True
    else:
        return False
def if_import(text):
    if re.search(r'import', text):
        return True
    else:
        return False
    
def if_null(text):
    if re.search(r'null', text):
        return True
    else:
        return False

def if_NIL(text):
    if re.search(r'NIL', text):
        return True
    else:
        return False
    
def if_void(text):
    if re.search(r'void', text):
        return True
    else:
        return False
    
def if_var(text):
    if re.search(r'var', text):
        return True
    else:
        return False
    
def if_function(text):
    if re.search(r'function', text):
        return True
    else:
        return False
    
def if_end(text):
    if re.search(r'end', text):
        return True
    else:
        return False
    
def avg_word_len(text):
    words = re.split('\W*', text)
    return sum([len(word) for word in words])/len(words)

def percentage_of_punctuation(text):
    total_length = len(text)
    text = re.sub(r'[\w\s]', '', text)
    punc_length = len(text)
    
    return punc_length / total_length

In [177]:
features = FunctionFeaturizer(paren_freq, thesis_freq, brac_freq,
                              ket_freq, lesser_freq, greater_freq,
                              slash_freq, dollar_freq, double_quote_freq,
                              space_freq, semi_freq, colon_freq, if_defn,
                              if_import, at_freq, star_freq, avg_word_len,
                              percentage_of_punctuation, if_null, if_NIL,
                              if_void, if_var, if_function, if_end,
                              pipe_freq)

In [178]:
lang_X_train, lang_X_test, lang_y_train, lang_y_test = train_test_split(lang_X, lang_y)

In [179]:
pipe = make_pipeline(features, DecisionTreeClassifier())
pipe.fit(lang_X_train, lang_y_train)
pipe.score(lang_X_train, lang_y_train)

1.0

In [180]:
pipe.score(lang_X_test, lang_y_test)

0.46341463414634149

In [58]:
print([1, 2, 3, 4])

[1, 2, 3, 4]
